In [1]:
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
%matplotlib inline

import sys
sys.path.append("/root/lib/ImageClassifier/")
import common_functions as my_functions

Using TensorFlow backend.


### Network last fullyconnected layer settings

In [2]:
num_classes = 17
last_activation = 'sigmoid'

In [3]:
import os
#run predict and save result
def run_predict(network_name, weights, dataset_path, flip='normal', rotate=''):
    result_path = '../predict_results/'+weights[:-3]+'_'+flip+'_'+str(rotate)+'_'+dataset_path+'.npz'
    print(result_path)

    if os.path.exists(result_path):
        ndarr = np.load(result_path)
        predict = ndarr['predict']
    else:
        ndarr = np.load(dataset_path)
        images = ndarr['test_images']
        dataset = my_functions.tta_images(images, flip, rotate)
        
        shape = (dataset.shape[1], dataset.shape[2], dataset.shape[3])
        model = eval('my_functions.'+network_name)(shape, num_classes, last_activation)
        model.load_weights(weights)
        predict = model.predict(dataset)
        np.savez(result_path, predict=predict)
    
    return predict

In [4]:
preditions = []

# 256px

In [23]:
weights = ['../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685.h5',
                    '../fullweights/0719_1407_ResNet50_256_14_0.0816_0.9685.h5',
                    '../fullweights/0719_1943_ResNet50_256_14_0.0818_0.9685.h5',
                    '../fullweights/0720_0840_ResNet50_256_14_0.0818_0.9684.h5',
                    '../fullweights/0720_1056_ResNet50_256_14_0.0816_0.9686.h5']

In [24]:
predictions_mean = []
for weight in weights:
    predictions = []
    sw = weight.split('_')
    cnn = sw[2]
    shape = sw[3]
    for flip in ['normal', 'lr', 'ud', 'udlr']:
        for rotate in ['normal', 90]:
            predictions.append( run_predict(cnn, weights=weight, dataset_path='test_'+str(shape)+'.npz', flip=flip, rotate=rotate) )
    predictions_mean.append(my_functions.mean_ensemble(predictions))
    
predictions_additional_mean = []
for weight in weights:
    predictions_additional = []
    sw = weight.split('_')
    cnn = sw[2]
    shape = sw[3]
    for flip in ['normal', 'lr', 'ud', 'udlr']:
        for rotate in ['normal', 90]:
            predictions_additional.append( run_predict(cnn, weights=weight, dataset_path='test_additional_'+str(shape)+'.npz', flip=flip, rotate=rotate) )
    predictions_additional_mean.append(my_functions.mean_ensemble(predictions_additional))
            
threshhold = []            
for weight in weights:
    for flip in ['normal', 'lr', 'ud', 'udlr']:
        for rotate in ['normal', 90]:
            result_path = weight+'_'+flip+'_'+str(rotate)+'_train_predict.npz'
            threshhold_npz = np.load(result_path)
            threshhold.append(threshhold_npz['threshhold'])
           


../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_normal_normal_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_normal_90_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_lr_normal_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_lr_90_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_ud_normal_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_ud_90_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_udlr_normal_test_256.npz.npz
../predict_results/../fullweights/0719_1206_ResNet50_256_14_0.0818_0.9685_udlr_90_test_256.npz.npz
../predict_results/../fullweights/0719_1407_ResNet50_256_14_0.0816_0.9685_normal_normal_test_256.npz.npz
../predict_results/../fullweights/0719_1407_ResNet50_256_14_0.0816_0.9685_normal_90_test_25

In [25]:
print(len(predictions_mean), len(predictions_additional_mean))

5 5


In [26]:
print(predictions_mean[0][0])
print(predictions_mean[1][0])
print(predictions_mean[2][0])
print(predictions_mean[3][0])
print(predictions_mean[4][0])


[  5.10507032e-03   9.99570847e-01   1.54092154e-02   9.96734701e-01
   5.93673272e-03   2.05813680e-03   6.40715915e-03   7.18768974e-03
   4.70113560e-04   8.02136797e-05   1.58962793e-03   2.55193397e-04
   7.69045808e-04   1.26249649e-04   9.16698473e-02   4.40895866e-03
   9.08961982e-04]
[  6.96678206e-03   9.99624155e-01   9.99520579e-03   9.95573312e-01
   6.64934292e-03   1.11823318e-03   4.16575653e-03   5.19929154e-03
   7.01210876e-04   2.30245211e-04   1.26087685e-03   8.75228789e-05
   1.53803945e-03   1.81656454e-04   1.10906595e-01   9.86098219e-03
   8.21796511e-04]
[  1.39334439e-02   9.98901159e-01   1.37068524e-02   9.93590266e-01
   5.90100538e-03   4.26372909e-03   3.70079066e-03   7.64819441e-03
   8.24606919e-04   1.57758509e-04   1.98762786e-03   1.74409964e-04
   1.89683688e-03   9.28877212e-05   9.51906769e-02   5.91230436e-03
   1.33738018e-03]
[  1.18707233e-02   9.98942710e-01   3.32846456e-02   9.89954248e-01
   6.95518643e-03   1.70940314e-03   1.1475830

In [27]:
predictions_mean2 = my_functions.mean_ensemble(predictions_mean)


In [28]:
print(len(predictions_mean2))

40669


In [29]:
predictions_mean = my_functions.mean_ensemble(predictions_mean)
predictions_additional_mean = my_functions.mean_ensemble(predictions_additional_mean)

In [30]:
print(len(predictions_mean), len(predictions_additional_mean))

40669 20522


# make submission data

In [31]:
labels = ['haze','primary', 'agriculture', 'clear', 'water', 'habitation', 'road',
 'cultivation','slash_burn', 'cloudy', 'partly_cloudy', 'conventional_mine',
 'bare_ground', 'artisinal_mine', 'blooming', 'selective_logging', 'blow_down']

In [32]:
thread =[0.25, 0.26, 0.23, 0.2, 0.19, 0.23, 0.3, 0.21, 0.21, 0.19, 0.2, 0.12, 0.17, 0.15, 0.28, 0.13, 0.09]

In [33]:
ndarr = np.load('test_256.npz')
file_names = ndarr['n']
n = []
for file_name in file_names:
    n.append(file_name)
    
ndarr = np.load('test_additional_256.npz')
file_names = ndarr['n']
for file_name in file_names:
    n.append(file_name)

In [34]:
print(len(predictions), len(predictions_additional), len(threshhold))

8 8 40


In [35]:
y = []
for i in range(len(predictions_mean)):
    y.append(predictions_mean[i])
for i in range(len(predictions_additional_mean)):
    y.append(predictions_additional_mean[i])
    
print(len(y))

61191


In [36]:
weather_label = ['haze', 'cloudy', 'partly_cloudy', 'clear' ]
r = []
for i in range(len(y)):  #len(y)
    y_label = ''
    
    test = y[i]
    for j in range(len(labels)):
        if test[j] > thread[j]:
            if y_label != '':
                y_label += ' '
            y_label += labels[j]
    r.append(y_label)



In [37]:
df = pd.DataFrame({'image_name':n, 'tags':r})
df.to_csv('submit.csv', index=False)

In [41]:
print(len(threshhold), len(predictions))

40 40


In [52]:
rows = []
for row in range( len(predictions[0]) ): #len(predictions)
    col = []
    for k in range(17):
        for i in range( len(predictions) ):
            pred = np.zeros(2)
            if predictions[i][row][k] > threshhold[i][k]:
                pred[1] += 1
            else:
                pred[0] += 1
        col.append(np.argmax(pred))
    rows.append(col)
    
    if row % 1000 == 0:
        print('loaded', row)
    


loaded 0
loaded 1000
loaded 2000
loaded 3000
loaded 4000
loaded 5000
loaded 6000
loaded 7000
loaded 8000
loaded 9000
loaded 10000
loaded 11000
loaded 12000
loaded 13000
loaded 14000
loaded 15000
loaded 16000
loaded 17000
loaded 18000
loaded 19000
loaded 20000
loaded 21000
loaded 22000
loaded 23000
loaded 24000
loaded 25000
loaded 26000
loaded 27000
loaded 28000
loaded 29000
loaded 30000
loaded 31000
loaded 32000
loaded 33000
loaded 34000
loaded 35000
loaded 36000
loaded 37000
loaded 38000
loaded 39000
loaded 40000


In [53]:
for row in range( len(predictions_additional[0]) ): #len(predictions)
    col = []
    for k in range(17):
        for i in range( len(predictions_additional) ):
            pred = np.zeros(2)
            if predictions_additional[i][row][k] > threshhold[i][k]:
                pred[1] += 1
            else:
                pred[0] += 1
        col.append(np.argmax(pred))
    rows.append(col)
    
    if row % 1000 == 0:
        print('loaded', row)
    

loaded 0
loaded 1000
loaded 2000
loaded 3000
loaded 4000
loaded 5000
loaded 6000
loaded 7000
loaded 8000
loaded 9000
loaded 10000
loaded 11000
loaded 12000
loaded 13000
loaded 14000
loaded 15000
loaded 16000
loaded 17000
loaded 18000
loaded 19000
loaded 20000


In [54]:
print( len(rows) )

61191


In [55]:
weather_label = ['haze', 'cloudy', 'partly_cloudy', 'clear' ]
r = []
for i in range(len(rows)):  #len(y)
    y_label = ''
    
    test = rows[i]
    for j in range(17):
        if test[j] == 1:
            if y_label != '':
                y_label += ' '
            y_label += labels[j]
    r.append(y_label)



In [56]:
df = pd.DataFrame({'image_name':n, 'tags':r})
df.to_csv('submit.csv', index=False)